In [2]:
import requests

In [3]:
GAME_STATUS_URL = 'https://draft.premierleague.com/api/game'
ELEMENT_INFO_URL = 'https://draft.premierleague.com/api/bootstrap-static'
LEAGUE_DATA_URL = 'https://draft.premierleague.com/api/league/{}/details'
ENTRY_PICKS_URL = 'https://draft.premierleague.com/api/entry/{}/event/{}'
ELEMENT_STATS_URL = 'https://draft.premierleague.com/api/event/{}/live'
TRANSFERS_URL = 'https://draft.premierleague.com/api/draft/league/{}/transactions'
DRAFTS_URL = 'https://draft.premierleague.com/api/draft/{}/choices'
MANAGER_HISTORY_URL = 'https://draft.premierleague.com/api/entry/{}/history'

In [4]:
league_id = 70382

In [5]:
entry_id = 339856

In [6]:
response = requests.get(GAME_STATUS_URL)
game_status = response.json()
current_gameweek = game_status['current_event']
current_gameweek

3

In [7]:
url = LEAGUE_DATA_URL.format(league_id)
response = requests.get(url)
league_data = response.json()
league_entries = league_data['league_entries']
standings = league_data['standings']
print(league_entries)
#print(standings)

[{'entry_id': 296815, 'entry_name': 'Democracy MANÉfest', 'id': 299237, 'joined_time': '2024-08-12T19:42:24.173380Z', 'player_first_name': 'Raife', 'player_last_name': 'Breton', 'short_name': 'RB', 'waiver_pick': 7}, {'entry_id': 336922, 'entry_name': 'Fridges Chilwell', 'id': 339856, 'joined_time': '2024-08-13T21:46:41.245484Z', 'player_first_name': 'Alex', 'player_last_name': 'McCafferty', 'short_name': 'AM', 'waiver_pick': 4}, {'entry_id': 337080, 'entry_name': 'Heungardium Levi-osa', 'id': 340018, 'joined_time': '2024-08-13T21:51:38.332131Z', 'player_first_name': 'Harry', 'player_last_name': 'Chapman', 'short_name': 'HC', 'waiver_pick': 1}, {'entry_id': 344796, 'entry_name': 'Belgian Bandits', 'id': 347884, 'joined_time': '2024-08-14T05:29:20.529921Z', 'player_first_name': 'Matthias', 'player_last_name': 'Vangool', 'short_name': 'MV', 'waiver_pick': 10}, {'entry_id': 344966, 'entry_name': 'David Gibheungminson', 'id': 348054, 'joined_time': '2024-08-14T05:44:12.353745Z', 'player_fi

In [8]:
url = MANAGER_HISTORY_URL.format(entry_id)
response = requests.get(url)
response.json()["history"]

[{'id': 181852,
  'points': 43,
  'total_points': 43,
  'rank': None,
  'rank_sort': None,
  'event_transfers': 0,
  'points_on_bench': 0,
  'entry': 339856,
  'event': 1},
 {'id': 575062,
  'points': 31,
  'total_points': 74,
  'rank': None,
  'rank_sort': None,
  'event_transfers': 0,
  'points_on_bench': 0,
  'entry': 339856,
  'event': 2},
 {'id': 1033807,
  'points': 22,
  'total_points': 96,
  'rank': None,
  'rank_sort': None,
  'event_transfers': 0,
  'points_on_bench': 0,
  'entry': 339856,
  'event': 3}]

In [9]:
# Function to get manager history
def get_manager_history(entry_id):
  url = MANAGER_HISTORY_URL.format(entry_id)
  response = requests.get(url)
  if response.status_code == 200:
      return response.json()["history"]
  else:
      print(f"Failed to get data for entry ID {entry_id}")
      return Non

In [24]:
# Create mapping from league_entry to entry_id
league_entry_to_entry_id = {entry['id']: entry['entry_id'] for entry in league_entries}
entry_id_to_name_to_team_map = {entry['entry_id']: (entry['player_first_name'], entry['entry_name']) for entry in league_entries}

print(league_entry_to_entry_id)
print(entry_id_to_name_to_team_map)

{299237: 296815, 339856: 336922, 340018: 337080, 347884: 344796, 348054: 344966, 368906: 365656, 394082: 390121, 410196: 406064, 412701: 408540, 417372: 413161}
{296815: ('Raife', 'Democracy MANÉfest'), 336922: ('Alex', 'Fridges Chilwell'), 337080: ('Harry', 'Heungardium Levi-osa'), 344796: ('Matthias', 'Belgian Bandits'), 344966: ('Johnny', 'David Gibheungminson'), 365656: ('Timothy', 'MatetaMakesYouWetta'), 390121: ('Will', 'Divockracy Manifest'), 406064: ('Ben', 'Botman McGinns'), 408540: ('Charlie', "Marie's Burning Bush"), 413161: ('Alex', 'Here we Guiu again')}


In [14]:
gameweek_scores = {entry['entry_id']: [] for entry in league_entries}
total_scores = {entry['entry_id']: [] for entry in league_entries}

print(gameweek_scores)
print(total_scores)

{296815: [], 336922: [], 337080: [], 344796: [], 344966: [], 365656: [], 390121: [], 406064: [], 408540: [], 413161: []}
{296815: [], 336922: [], 337080: [], 344796: [], 344966: [], 365656: [], 390121: [], 406064: [], 408540: [], 413161: []}


In [15]:
# Process each entry ID
for entry_id in gameweek_scores.keys():
  history = get_manager_history(entry_id)
  
  if history:
      # Sort the history by event number
      sorted_history = sorted(history, key=lambda x: x['event'])
      
      # Initialize event counter
      current_event = 1
      
      for event in sorted_history:
          # Fill in any missing events with None
          while current_event < event['event']:
              gameweek_scores[entry_id].append(None)
              total_scores[entry_id].append(None)
              current_event += 1
          
          # Append the actual points and total_points
          gameweek_scores[entry_id].append(event['points'])
          total_scores[entry_id].append(event['total_points'])
          current_event += 1

In [16]:
print(gameweek_scores)
print(total_scores)

{296815: [38, 52, 50], 336922: [41, 29, 29], 337080: [24, 28, 23], 344796: [63, 32, 52], 344966: [34, 44, 30], 365656: [33, 39, 25], 390121: [30, 35, 19], 406064: [44, 63, 35], 408540: [39, 45, 38], 413161: [47, 51, 48]}
{296815: [38, 90, 140], 336922: [41, 70, 99], 337080: [24, 52, 75], 344796: [63, 95, 147], 344966: [34, 78, 108], 365656: [33, 72, 97], 390121: [30, 65, 84], 406064: [44, 107, 142], 408540: [39, 84, 122], 413161: [47, 98, 146]}


In [19]:
print([entry_id_to_name_map[296815]])

['Raife']
